In [18]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pyttsx3
mp_drawing = mp.solutions.drawing_utils #my notes=  to visulize the lines on the body 
mp_pose = mp.solutions.pose # from solutions we want the pose estemation model

In [29]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0]) #y2-y1 , x2-x1 , here we calculated the dgree in radians
    angle = np.abs(radians*180.0/np.pi) # we converted radiance to dgree
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [50]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate angle (not used here, but you can keep it for reference)
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arccos(
        np.clip(np.dot(a - b, c - b) / (np.linalg.norm(a - b) * np.linalg.norm(c - b)), -1.0, 1.0)
    )
    return np.degrees(radians)

cap = cv2.VideoCapture(0)

counter = 0
stage = "center"

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # Flip the image to mirror the webcam feed

        # Convert the image to RGB (required by MediaPipe)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of relevant landmarks
            NOSE = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                    landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            # Midpoint of the shoulders (used for head direction)
            shoulder_mid_x = (LEFT_SHOULDER[0] + RIGHT_SHOULDER[0]) / 2
            threshold = 0.15  # Adjust this value for sensitivity

            # Detect direction based on nose's x-position
            if stage == "center":
                if NOSE[0] < (shoulder_mid_x - threshold):
                    stage = "left"  # Head turned left
                elif NOSE[0] > (shoulder_mid_x + threshold):
                    stage = "right"  # Head turned right

            elif stage in ["left", "right"]:
                if (shoulder_mid_x - threshold) <= NOSE[0] <= (shoulder_mid_x + threshold):
                    stage = "center"  # Head back to center
                    counter += 1  # Increment rep count
                    print("Reps:", counter)

            # Debug: Draw the midpoint line and nose
            mid_x_pixel = int(shoulder_mid_x * 640)  # Convert to pixel coordinates
            #cv2.line(image, (mid_x_pixel, 0), (mid_x_pixel, 480), (0, 255, 0), 2)  # Midpoint line

            nose_pixel = tuple(np.multiply(NOSE, [640, 480]).astype(int))  # Nose pixel
            cv2.circle(image, nose_pixel, 5, (255, 0, 0), -1)  # Draw nose position

        except:
            pass

        # Counter box for displaying rep count
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)

        # Display rep count
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        # Display stage (center, left, right)
        cv2.putText(image, 'STAGE', (65, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, stage, (60, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        # Draw pose landmarks for visual feedback
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Show the webcam feed with the annotations
        cv2.imshow('Mediapipe Feed', image)

        # Break the loop when 'q' key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Reps: 1
Reps: 2
Reps: 3
Reps: 4


i added a full reb cycle " moving from left/right to right/left " 

In [56]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate angle (not used here, but you can keep it for reference)
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arccos(
        np.clip(np.dot(a - b, c - b) / (np.linalg.norm(a - b) * np.linalg.norm(c - b)), -1.0, 1.0)
    )
    return np.degrees(radians)

cap = cv2.VideoCapture(0)

left_count = 0
center_count = 0
right_count = 0
cycle_count = 0  # Track number of complete cycles
stage = "center"

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # Flip the image to mirror the webcam feed

        # Convert the image to RGB (required by MediaPipe)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of relevant landmarks
            NOSE = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                    landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            # Midpoint of the shoulders (used for head direction)
            shoulder_mid_x = (LEFT_SHOULDER[0] + RIGHT_SHOULDER[0]) / 2
            threshold = 0.15  # Adjust this value for sensitivity

            # Detect direction based on nose's x-position
            if stage == "center":
                if NOSE[0] < (shoulder_mid_x - threshold):
                    stage = "left"  # Head turned left
                    left_count += 1  # Increment left count
                elif NOSE[0] > (shoulder_mid_x + threshold):
                    stage = "right"  # Head turned right
                    right_count += 1  # Increment right count

            elif stage == "left":
                if (shoulder_mid_x - threshold) <= NOSE[0] <= (shoulder_mid_x + threshold):
                    stage = "center"  # Head back to center
                    center_count += 1  # Increment center count

            elif stage == "right":
                if (shoulder_mid_x - threshold) <= NOSE[0] <= (shoulder_mid_x + threshold):
                    stage = "center"  # Head back to center
                    center_count += 1  # Increment center count

            # If total counts for left, center, and right reach 3, we reset the counts
            total_count = left_count + center_count + right_count
            if total_count == 4:
                cycle_count += 1  # Increment cycle count
                left_count = 0  # Reset left count
                center_count = 0  # Reset center count
                right_count = 0  # Reset right count

        except:
            pass

        # Display the movement counts and cycle count
        cv2.rectangle(image, (0, 0), (225, 200), (245, 117, 16), -1)

        # Display left, center, right counts and cycle count
        cv2.putText(image, 'Left: ' + str(left_count), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, 'Center: ' + str(center_count), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, 'Right: ' + str(right_count), (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, 'Cycles: ' + str(cycle_count), (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        # Draw pose landmarks for visual feedback
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Show the webcam feed with the annotations
        cv2.imshow('Mediapipe Feed', image)

        # Break the loop when 'q' key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


i added timer

In [10]:
import cv2
import mediapipe as mp
import numpy as np
import time  # Import the time module

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate angle (not used here, but you can keep it for reference)
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arccos(
        np.clip(np.dot(a - b, c - b) / (np.linalg.norm(a - b) * np.linalg.norm(c - b)), -1.0, 1.0)
    )
    return np.degrees(radians)

cap = cv2.VideoCapture(0)

left_count = 0
center_count = 0
right_count = 0
cycle_count = 0  # Track number of complete cycles
stage = "center"

# Variables for the countdown timer
countdown_time = 5  
countdown_started = False
countdown_end_time = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # Flip the image to mirror the webcam feed

        # Convert the image to RGB (required by MediaPipe)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of relevant landmarks
            NOSE = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                    landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            # Midpoint of the shoulders (used for head direction)
            shoulder_mid_x = (LEFT_SHOULDER[0] + RIGHT_SHOULDER[0]) / 2
            threshold = 0.15  # Adjust this value for sensitivity

            # Detect direction based on nose's x-position
            if stage == "center":
                if NOSE[0] < (shoulder_mid_x - threshold):
                    stage = "left"  # Head turned left
                    left_count += 1  # Increment left count
                    countdown_started = True
                    countdown_end_time = time.time() + countdown_time  # Start countdown

                elif NOSE[0] > (shoulder_mid_x + threshold):
                    stage = "right"  # Head turned right
                    right_count += 1  # Increment right count
                    countdown_started = True
                    countdown_end_time = time.time() + countdown_time  # Start countdown

            elif stage == "left":
                if (shoulder_mid_x - threshold) <= NOSE[0] <= (shoulder_mid_x + threshold):
                    stage = "center"  # Head back to center
                    center_count += 1  # Increment center count
                    countdown_started = False  # Stop countdown when moving back to center

            elif stage == "right":
                if (shoulder_mid_x - threshold) <= NOSE[0] <= (shoulder_mid_x + threshold):
                    stage = "center"  # Head back to center
                    center_count += 1  # Increment center count
                    countdown_started = False  # Stop countdown when moving back to center

            # If total counts for left, center, and right reach 3, we reset the counts
            total_count = left_count + center_count + right_count
            if total_count == 4:
                cycle_count += 1  # Increment cycle count
                left_count = 0  # Reset left count
                center_count = 0  # Reset center count
                right_count = 0  # Reset right count

        except:
            pass

        # Display the movement counts and cycle count
        cv2.rectangle(image, (0, 0), (225, 200), (245, 117, 16), -1)

        # Display left, center, right counts and cycle count
        cv2.putText(image, 'Left: ' + str(left_count), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, 'Center: ' + str(center_count), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, 'Right: ' + str(right_count), (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, 'Cycles: ' + str(cycle_count), (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        # Show countdown timer if moving left or right
        if countdown_started:
            remaining_time = int(countdown_end_time - time.time())
            if remaining_time > 0:
                minutes = remaining_time // 60
                seconds = remaining_time % 60
                countdown_text = f"{minutes:02}:{seconds:02}"
            else:
                countdown_text = "00:00"  # Countdown finished
            cv2.putText(image, f"Countdown: {countdown_text}", (10, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        # Draw pose landmarks for visual feedback
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Show the webcam feed with the annotations
        cv2.imshow('Mediapipe Feed', image)

        # Break the loop when 'q' key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


what to do next : 1- we should add voices , 2- we should adjuct the code for the cycles 

1- here i will adjuct the code for the cycle " the countdown will restart if there is a change in the users neck dirction"

In [16]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

left_count = 0
center_count = 0
right_count = 0
cycle_count = 0
stage = "center"

countdown_time = 5
countdown_start_time = None
countdown_direction = None  # "left" or "right"

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            NOSE = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                    landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            shoulder_mid_x = (LEFT_SHOULDER[0] + RIGHT_SHOULDER[0]) / 2
            threshold = 0.15
            buffer = 0.03  # Tolerance for slight movement
            current_time = time.time()

            in_left_zone = NOSE[0] < (shoulder_mid_x - threshold)
            in_right_zone = NOSE[0] > (shoulder_mid_x + threshold)
            in_center_zone = not in_left_zone and not in_right_zone

            # LEFT
            if in_left_zone:
                if countdown_direction != "left":
                    countdown_start_time = current_time
                    countdown_direction = "left"
                else:
                    elapsed = current_time - countdown_start_time
                    if elapsed >= countdown_time and stage != "left":
                        stage = "left"
                        left_count += 1
                        countdown_direction = None

            # RIGHT
            elif in_right_zone:
                if countdown_direction != "right":
                    countdown_start_time = current_time
                    countdown_direction = "right"
                else:
                    elapsed = current_time - countdown_start_time
                    if elapsed >= countdown_time and stage != "right":
                        stage = "right"
                        right_count += 1
                        countdown_direction = None

            # CENTER
            elif in_center_zone:
                if stage in ["left", "right"]:
                    center_count += 1
                    stage = "center"
                countdown_start_time = None
                countdown_direction = None

            # Buffer-based reset
            if countdown_direction == "left":
                if NOSE[0] > (shoulder_mid_x - threshold + buffer):
                    countdown_start_time = None
                    countdown_direction = None

            elif countdown_direction == "right":
                if NOSE[0] < (shoulder_mid_x + threshold - buffer):
                    countdown_start_time = None
                    countdown_direction = None

            # Count a cycle
            if left_count > 0 and right_count > 0 and center_count > 1:
                cycle_count += 1
                left_count = 0
                right_count = 0
                center_count = 0

        except:
            pass

        # Display data
        cv2.rectangle(image, (0, 0), (250, 200), (245, 117, 16), -1)
        cv2.putText(image, f'Left: {left_count}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, f'Center: {center_count}', (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, f'Right: {right_count}', (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, f'Cycles: {cycle_count}', (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        if countdown_start_time is not None and countdown_direction is not None:
            remaining = countdown_time - int(current_time - countdown_start_time)
            if remaining > 0:
                cv2.putText(image, f'{countdown_direction.capitalize()} Hold: 00:{remaining:02}',
                            (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
            else:
                cv2.putText(image, f'{countdown_direction.capitalize()} Hold: Done',
                            (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


there is a problem : the count will count again if the user didnt move his head , here it fixed: 

In [19]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

left_count = 0
center_count = 0
right_count = 0
cycle_count = 0
stage = "center"

countdown_time = 5
countdown_start_time = None
countdown_direction = None  # "left" or "right"

# Flags to ensure one count per side before resetting
left_ready = True
right_ready = True

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            NOSE = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                    landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            shoulder_mid_x = (LEFT_SHOULDER[0] + RIGHT_SHOULDER[0]) / 2
            threshold = 0.15
            buffer = 0.03  # Tolerance for slight movement
            current_time = time.time()

            in_left_zone = NOSE[0] < (shoulder_mid_x - threshold)
            in_right_zone = NOSE[0] > (shoulder_mid_x + threshold)
            in_center_zone = not in_left_zone and not in_right_zone

            # LEFT
            if in_left_zone and left_ready:
                if countdown_direction != "left":
                    countdown_start_time = current_time
                    countdown_direction = "left"
                else:
                    elapsed = current_time - countdown_start_time
                    if elapsed >= countdown_time and stage != "left":
                        stage = "left"
                        left_count += 1
                        countdown_direction = None
                        left_ready = False  # Prevent re-triggering

            # RIGHT
            elif in_right_zone and right_ready:
                if countdown_direction != "right":
                    countdown_start_time = current_time
                    countdown_direction = "right"
                else:
                    elapsed = current_time - countdown_start_time
                    if elapsed >= countdown_time and stage != "right":
                        stage = "right"
                        right_count += 1
                        countdown_direction = None
                        right_ready = False  # Prevent re-triggering

            # CENTER
            elif in_center_zone:
                if stage in ["left", "right"]:
                    center_count += 1
                    stage = "center"
                    left_ready = True
                    right_ready = True
                countdown_start_time = None
                countdown_direction = None

            # Buffer-based reset
            if countdown_direction == "left":
                if NOSE[0] > (shoulder_mid_x - threshold + buffer):
                    countdown_start_time = None
                    countdown_direction = None

            elif countdown_direction == "right":
                if NOSE[0] < (shoulder_mid_x + threshold - buffer):
                    countdown_start_time = None
                    countdown_direction = None

            # Count a cycle
            if left_count > 0 and right_count > 0 and center_count > 1:
                cycle_count += 1
                left_count = 0
                right_count = 0
                center_count = 0

        except:
            pass

        # Display data
        cv2.rectangle(image, (0, 0), (250, 200), (245, 117, 16), -1)
        cv2.putText(image, f'Left: {left_count}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, f'Center: {center_count}', (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, f'Right: {right_count}', (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, f'Cycles: {cycle_count}', (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        if countdown_start_time is not None and countdown_direction is not None:
            remaining = countdown_time - int(current_time - countdown_start_time)
            if remaining > 0:
                cv2.putText(image, f'{countdown_direction.capitalize()} Hold: 00:{remaining:02}',
                            (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
            else:
                cv2.putText(image, f'{countdown_direction.capitalize()} Hold: Done',
                            (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


2- i will add a voice

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

left_count = 0
center_count = 0
right_count = 0
cycle_count = 0
stage = "center"

countdown_time = 5
countdown_start_time = None
countdown_direction = None  # "left" or "right"

# Flags to ensure one count per side before resetting
left_ready = True
right_ready = True

def reset_countdown(message=None):
    global countdown_start_time, countdown_direction
    if countdown_start_time is not None and countdown_direction is not None and message:
        print(message)
    countdown_start_time = None
    countdown_direction = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            NOSE = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                    landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            shoulder_mid_x = (LEFT_SHOULDER[0] + RIGHT_SHOULDER[0]) / 2
            threshold = 0.15
            buffer = 0.05  # Tolerance for slight movement
            current_time = time.time()

            in_left_zone = NOSE[0] < (shoulder_mid_x - threshold)
            in_right_zone = NOSE[0] > (shoulder_mid_x + threshold)
            in_center_zone = not in_left_zone and not in_right_zone

            # LEFT
            if in_left_zone and left_ready:
                if countdown_direction != "left":
                    countdown_start_time = current_time
                    countdown_direction = "left"
                    print("Started countdown for LEFT hold...")
                else:
                    elapsed = current_time - countdown_start_time
                    if elapsed >= countdown_time and stage != "left":
                        stage = "left"
                        left_count += 1
                        print("✅ LEFT hold completed!")
                        reset_countdown()
                        left_ready = False

            # RIGHT
            elif in_right_zone and right_ready:
                if countdown_direction != "right":
                    countdown_start_time = current_time
                    countdown_direction = "right"
                    print("Started countdown for RIGHT hold...")
                else:
                    elapsed = current_time - countdown_start_time
                    if elapsed >= countdown_time and stage != "right":
                        stage = "right"
                        right_count += 1
                        print("✅ RIGHT hold completed!")
                        reset_countdown()
                        right_ready = False

            # CENTER
            elif in_center_zone:
                if stage in ["left", "right"]:
                    center_count += 1
                    stage = "center"
                    left_ready = True
                    right_ready = True
                    print("Returned to CENTER")
                reset_countdown(f"❌ {countdown_direction.upper()} hold interrupted — exited the zone early.")

            # Buffer-based reset
            if countdown_direction == "left":
                if NOSE[0] > (shoulder_mid_x - threshold + buffer):
                    reset_countdown("❌ LEFT hold interrupted — exited the zone early.")

            elif countdown_direction == "right":
                if NOSE[0] < (shoulder_mid_x + threshold - buffer):
                    reset_countdown("❌ RIGHT hold interrupted — exited the zone early.")

            # Count a cycle
            if left_count > 0 and right_count > 0 and center_count > 1:
                cycle_count += 1
                left_count = 0
                right_count = 0
                center_count = 0
                print("🔁 Full cycle completed!")

        except:
            pass

        # Display data
        cv2.rectangle(image, (0, 0), (250, 200), (245, 117, 16), -1)
        cv2.putText(image, f'Left: {left_count}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, f'Center: {center_count}', (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, f'Right: {right_count}', (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, f'Cycles: {cycle_count}', (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        if countdown_start_time is not None and countdown_direction is not None:
            remaining = countdown_time - int(current_time - countdown_start_time)
            if remaining > 0:
                cv2.putText(image, f'{countdown_direction.capitalize()} Hold: 00:{remaining:02}',
                            (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
            else:
                cv2.putText(image, f'{countdown_direction.capitalize()} Hold: Done',
                            (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Started countdown for LEFT hold...
✅ LEFT hold completed!
Returned to CENTER
Started countdown for LEFT hold...
❌ LEFT hold interrupted — exited the zone early.


i will ad the voice here

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pyttsx3
import threading
import queue

# Initialize the voice engine
engine = pyttsx3.init()

# Set the rate (speed) of the speech
engine.setProperty('rate', 150)  # Adjust as needed

# Set the volume (0.0 to 1.0)
engine.setProperty('volume', 1.0)

# Set voice (0 for male, 1 for female)
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)  # Female voice, you can change this index

# Initialize the drawing and pose model
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

# Frame processing frequency (process every nth frame)
frame_skip = 2
frame_count = 0

left_count = 0
center_count = 0
right_count = 0
cycle_count = 0
stage = "center"

countdown_time = 5
countdown_start_time = None
countdown_direction = None  # "left" or "right"

# Flags to ensure one count per side before resetting
left_ready = True
right_ready = True

# Queue for messages to speak
speech_queue = queue.Queue()

def speak_message(message):
    """Add the message to the speech queue."""
    speech_queue.put(message)

def process_speech():
    """Process the speech queue in a separate thread."""
    while True:
        if not speech_queue.empty():
            message = speech_queue.get()
            engine.say(message)
            engine.runAndWait()

# Start the speech processing thread
speech_thread = threading.Thread(target=process_speech, daemon=True)
speech_thread.start()

def reset_countdown(message=None):
    global countdown_start_time, countdown_direction
    if countdown_start_time is not None and countdown_direction is not None and message:
        speak_message(message)  # Queue the message for speaking
    countdown_start_time = None
    countdown_direction = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue  # Skip if no frame is grabbed

        frame_count += 1
        if frame_count % frame_skip != 0:
            continue  # Skip processing some frames to improve performance

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            NOSE = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                    landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            shoulder_mid_x = (LEFT_SHOULDER[0] + RIGHT_SHOULDER[0]) / 2
            threshold = 0.15
            buffer = 0.03  # Tolerance for slight movement
            current_time = time.time()

            in_left_zone = NOSE[0] < (shoulder_mid_x - threshold)
            in_right_zone = NOSE[0] > (shoulder_mid_x + threshold)
            in_center_zone = not in_left_zone and not in_right_zone

            # LEFT
            if in_left_zone and left_ready:
                if countdown_direction != "left":
                    countdown_start_time = current_time
                    countdown_direction = "left"
                    speak_message("Started countdown for LEFT hold...")
                else:
                    elapsed = current_time - countdown_start_time
                    if elapsed >= countdown_time and stage != "left":
                        stage = "left"
                        left_count += 1
                        speak_message("LEFT hold completed!")
                        reset_countdown()
                        left_ready = False

            # RIGHT
            elif in_right_zone and right_ready:
                if countdown_direction != "right":
                    countdown_start_time = current_time
                    countdown_direction = "right"
                    speak_message("Started countdown for RIGHT hold...")
                else:
                    elapsed = current_time - countdown_start_time
                    if elapsed >= countdown_time and stage != "right":
                        stage = "right"
                        right_count += 1
                        speak_message("RIGHT hold completed!")
                        reset_countdown()
                        right_ready = False

            # CENTER
            elif in_center_zone:
                if stage in ["left", "right"]:
                    center_count += 1
                    stage = "center"
                    left_ready = True
                    right_ready = True
                    speak_message("Returned to CENTER")
                reset_countdown(f" {countdown_direction.upper()} you moved, stay still.")

            # Buffer-based reset
            if countdown_direction == "left":
                if NOSE[0] > (shoulder_mid_x - threshold + buffer):
                    reset_countdown("you moved, stay still.")

            elif countdown_direction == "right":
                if NOSE[0] < (shoulder_mid_x + threshold - buffer):
                    reset_countdown("you moved, stay still.")

            # Count a cycle
            if left_count > 0 and right_count > 0 and center_count > 1:
                cycle_count += 1
                left_count = 0
                right_count = 0
                center_count = 0
                speak_message("🔁 Full cycle completed!")

        except:
            pass

        # Display data
        cv2.rectangle(image, (0, 0), (250, 200), (245, 117, 16), -1)
        cv2.putText(image, f'Left: {left_count}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, f'Center: {center_count}', (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, f'Right: {right_count}', (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, f'Cycles: {cycle_count}', (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        if countdown_start_time is not None and countdown_direction is not None:
            remaining = countdown_time - int(current_time - countdown_start_time)
            if remaining > 0:
                cv2.putText(image, f'{countdown_direction.capitalize()} Hold: 00:{remaining:02}',
                            (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
            else:
                cv2.putText(image, f'{countdown_direction.capitalize()} Hold: Done',
                            (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
